In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn

seaborn.set_style("darkgrid")
plt.rc("figure", figsize=(16, 10))
plt.rc("savefig", dpi=90)
plt.rc("font", family="sans-serif")
plt.rc("font", size=14)

In [11]:
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://localhost', connect_args={'read_default_file': '~/.mytest.cnf'})
conn = engine.connect()

In [12]:
import pandas as pd

SQL_query = pd.read_sql_query(
    '''select tradedate, pct_close from useq.USEQ_HIST where symbol='MS'
    and volume>0 order by tradedate''',
    conn,
    index_col='tradedate'
)

df = pd.DataFrame(SQL_query, columns=['pct_close'])

df.head()


ProgrammingError: (pymysql.err.ProgrammingError) (1146, "Table 'useq.USEQ_HIST' doesn't exist")
[SQL: select tradedate, pct_close from useq.USEQ_HIST where symbol='MS'
    and volume>0 order by tradedate]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
ax = df.plot()
xlim = ax.set_xlim(df.index.min(), df.index.max())


In [ ]:
from arch import arch_model

am = arch_model(df, rescale=False, p=1, o=1, q=1, dist="StudentsT")
resg = am.fit(update_freq=5)
print(resg.summary())

In [ ]:
#fig = res.plot(annualize="D")

In [ ]:
from arch.univariate import ARX, HARX, HARCH, GARCH, StudentsT, SkewStudent

am=ARX(df, rescale=False, lags=[1,5,22], constant=True)
am.volatility = GARCH(1,1,1)
am.distribution = StudentsT()
resx = am.fit(update_freq=5)
print(resx.summary())

In [ ]:
am.distribution = SkewStudent()
resk = am.fit(update_freq=5)
print(resk.summary())

In [ ]:
plt.plot(resg.std_resid, label='GJR')
plt.plot(resx.std_resid, label='ARX')

In [ ]:
ax = plt.plot(resg.conditional_volatility, label='GJR')
plt.plot(resx.conditional_volatility, label='ARX')

In [ ]:
am.volatility = HARCH(lags=[1,5,22])
am.distribution = StudentsT()
res = am.fit(update_freq=5)
print(res.summary())
am.distribution = SkewStudent()
res = am.fit(update_freq=5)
print(res.summary())

In [ ]:
print("Log-Likelihood = ", resg.loglikelihood, " for GJR")
print("Log-Likelihood = ", resx.loglikelihood, " for ARX")
print("nobs = ", resg.nobs, " for GJR")
print ("nobs = ", resx.nobs, " for ARX")
factor = resg.nobs / resx.nobs
adjusted = resx.loglikelihood * factor
print("Log-Likelihood adjusted = ", adjusted, " for ARX")
